# River bifurcation in CONUS workflow
This notebook contains the workflow necessary to extract data from a HUC4 and join it to NABD for bifurcation analysis.

## 1. Install packages

In [ ]:
# import sys
# !{sys.executable} -m pip install geofeather
# !{sys.executable} -m pip install nhdnet  #see Setup info document 

## 2. Load modules

In [2]:
#basic analysis 
from pathlib import Path
import os
from time import time
import geopandas as gp
import geofeather
import numpy as np
import pandas as pd


#modules from SARP analysis
from geofeather import to_geofeather
from nhdnet.nhd.extract import extract_flowlines 
# from nhdnet.nhd.extract_test import extract_flowlines_R  # this function was created by Rachel to extract info from other NHD
# from nhdnet.nhd.extract import extract_waterbodies
from nhdnet.io import serialize_df, serialize_sindex, to_shp 

#Getting the other NHD 
# from nhdnet.nhd.download import download_huc4
# nhd_dir = Path("data/nhd/source/huc4")


# #pull data from Google Drive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive

# gauth = GoogleAuth()
# gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
# drive = GoogleDrive(gauth)

## 3. Initial setup and constants

In [3]:
#Select HUC of interest
HUC2 = 10
i = 19
HUC4 = "{0}{1:02d}".format(HUC2, i)  # this formats the HUC4 name how we want it. ':02d' is string formatting
print(HUC4)
print(type(HUC4))
huc_id = int(HUC4) * 1000000   # the full HUC4 ID
print(huc_id)

data_dir = Path("data/nhd/source/huc4")  # point to where GDBs are
# data_dir = Path("/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/GIS/Layers/NHDPlusNationalData")  # point to where GDBs are

#Setting projections
CRS = {           # Using USGS CONUS Albers (EPSG:102003): https://epsg.io/102003  WHY?
    "proj": "aea",
    "lat_1": 29.5,
    "lat_2": 45.5,
    "lat_0": 37.5,
    "lon_0": -96,
    "x_0": 0,
    "y_0": 0,
    "datum": "NAD83",
    "units": "m",
    "no_defs": True,
}
print(data_dir)

1019
<class 'str'>
1019000000
data/nhd/source/huc4


## 4. Read in the geodatabase

In [4]:
# sys.path.append('/Users/rachelspinti/Documents/River_bifurcation/data/nhd/source/huc4/1019') #call where these scripts are located

gdb = data_dir/HUC4/ "NHDPLUS_H_{HUC4}_HU4_GDB.gdb".format(HUC4=HUC4)
print(gdb)
read_start = time()
flowlines, joins = extract_flowlines(gdb, target_crs=CRS)
print("Read {:,} flowlines in  {:.0f} seconds".format(len(flowlines), time() - read_start))

# gdb = data_dir/"NHDPlusV21_National_Seamless_Flattened_Lower48.gdb"
# print(gdb)
# read_start = time()
# flowlines = extract_flowlines_R(gdb, target_crs=CRS)
# print("Read {:,} flowlines in  {:.0f} seconds".format(len(flowlines), time() - read_start))

# gdb = data_dir/ "NHDPlusV21_National_Seamless_Flattened_Lower48.gdb".format(HUC4=HUC4)
# print(gdb)
# read_start = time()
# flowlines, joins = extract_flowlines(gdb, target_crs=CRS)
# print("Read {:,} flowlines in  {:.0f} seconds".format(len(flowlines), time() - read_start))


data/nhd/source/huc4/1019/NHDPLUS_H_1019_HU4_GDB.gdb
Reading flowlines
Columns= <bound method NDFrame.head of            NHDPlusID  FlowDir  FType   GNIS_ID             GNIS_Name  \
0       2.300190e+13        1    460      None                  None   
1       2.300190e+13        1    460      None                  None   
2       2.300190e+13        1    460      None                  None   
3       2.300190e+13        1    460      None                  None   
4       2.300190e+13        1    460      None                  None   
...              ...      ...    ...       ...                   ...   
232506  2.300190e+13        1    460      None                  None   
232507  2.300190e+13        1    460  00184959  West Toll Gate Creek   
232508  2.300190e+13        1    334      None                  None   
232509  2.300190e+13        1    558      None                  None   
232510  2.300190e+13        1    336  00203030      Burlington Ditch   

                         

### 4.1 Some checks and cleanup of 'flowlines'

In [5]:
# flowlines.head(3)
# list(flowlines.columns)

# cleaning up columns
imp_cols = ["lineID", "NHDPlusID", "ReachCode", "streamorder", "geometry"]
flowlines = flowlines[imp_cols]
list(flowlines.columns)
# len(flowlines.columns)

['lineID', 'NHDPlusID', 'ReachCode', 'streamorder', 'geometry']

### 4.2 Read in the HUC4 shapefile (1019) I made 

In [6]:
huc_test = (gp.read_file('/Users/rachelspinti/Desktop/HUC_test/Test1029.shp')) # this is actually HUC 1019
huc_test = huc_test.rename(columns={"REACHCODE": "ReachCode"})
huc_test.ReachCode = huc_test.ReachCode.astype("uint64")
# print(huc_test)

### 4.3 Checks and cleanup of 'huc_test'

In [7]:
# checking huc_test
# type(huc_test.ReachCode)
# huc_test.head(3)
# list(huc_test.columns)
print(len(huc_test))
# cleaning up columns
imp_cols2 = ['OBJECTID', 'COMID', 'ReachCode']
huc_test = huc_test[imp_cols2]
# huc_test.drop(huc_test.columns[[3, 4, 5, 6,7 ]], axis=1, inplace=True)
list(huc_test.columns)
# # len(huc_test.columns)


14847


['OBJECTID', 'COMID', 'ReachCode']

In [8]:
len(flowlines)

221415

## 5. Join the two datasets
Check this link out for help: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/vector-data-processing/spatial-joins-in-python-geopandas-shapely/

See also: https://geopandas.org/mergingdata.html

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [9]:
# Attribute join with geopandas
rivers = flowlines.merge(huc_test, how='left', on='ReachCode')

### 5.1 Checks of 'rivers'

In [10]:
# Checking stuff after merge
print(len(rivers))
print(rivers.head(4))  #check the join
# print(rivers.COMID)
# rivers.iloc[rivers.streamorder].plot()
# flowlines[flowlines.streamorder>4].plot()

# Create a list of unique values by turning the
# pandas column into a set
# len(set(rivers.ReachCode))
len(set(rivers.COMID))

# type(rivers)
# print(rivers.describe)
# rivers.plot()
# print(rivers.shape)
# print(list(rivers.columns))

246765
   lineID       NHDPlusID       ReachCode  streamorder  \
0       1  23001900008851  10190001010972            1   
1       2  23001900008863  10190002021576            1   
2       3  23001900008864  10190002022431            1   
3       4  23001900008865  10190002022326            1   

                                            geometry  OBJECTID  COMID  
0  LINESTRING (-800081.762 215515.782, -800082.87...       NaN    NaN  
1  LINESTRING (-799823.525 221309.940, -799829.56...       NaN    NaN  
2  LINESTRING (-800259.206 220461.613, -800252.86...       NaN    NaN  
3  LINESTRING (-799873.180 220328.121, -799869.36...       NaN    NaN  


176075

## 6. Join NHD and NABD

In [11]:
# Read in NABD shapefile and join with NHD
nabd = gp.read_file('/Users/rachelspinti/Documents/River_bifurcation/data/nabd/nabd_fish_barriers_2012.shp')
imp_cols3 = ['NIDID', 'COMID', 'Dam_name', 'Purposes', 'Year_compl', 'Max_stor', 'Norm_stor','NID_stor', 'geometry']
nabd = nabd[imp_cols3]
print(list(nabd.columns))

['NIDID', 'COMID', 'Dam_name', 'Purposes', 'Year_compl', 'Max_stor', 'Norm_stor', 'NID_stor', 'geometry']


In [12]:
# Merge
nabd = nabd.merge(rivers, on='COMID')   #attribute join with geopandas

In [13]:
# Checking stuff after merge
print(len(nabd))
nabd.head(4)  #check the join
# nabd[nabd.COMID].plot()

2073


,NIDID,COMID,Dam_name,Purposes,Year_compl,Max_stor,Norm_stor,NID_stor,geometry_x,lineID,NHDPlusID,ReachCode,streamorder,geometry_y,OBJECTID
0,CO00307,188119.0,SMITH,IR,1884.0,822.0,638.0,822.0,POINT (-105.11384 39.68576),172230,23001900133800,10190002000457,2,"LINESTRING (-773431.077 282105.494, -773395.55...",1520138.0
1,CO00297,188127.0,KENDRICK,IR,1922.0,492.0,242.0,492.0,POINT (-105.10443 39.68396),73964,23001900047389,10190002000455,2,"LINESTRING (-769177.591 281385.829, -769166.58...",1520132.0
2,CO00297,188127.0,KENDRICK,IR,1922.0,492.0,242.0,492.0,POINT (-105.10443 39.68396),73965,23001900047390,10190002000455,2,"LINESTRING (-768913.600 281410.653, -768770.37...",1520132.0
3,CO00297,188127.0,KENDRICK,IR,1922.0,492.0,242.0,492.0,POINT (-105.10443 39.68396),74291,23001900046500,10190002000455,2,"LINESTRING (-772040.876 281657.221, -771962.28...",1520132.0
